In [62]:
import os
import numpy as np
import pandas as pd

In [63]:
res_path = './res'
feat_path = './features'
train_path = './data/train'
labels = os.listdir(train_path)

In [64]:
linux_test_order = np.argsort(np.load('./linux_order_test.npy'))
linux_label_order = np.argsort(np.load('./linux_order_label.npy'))

In [73]:
model_names = ['densenet', 'resnest50', 'resnet50', 'cnn_base', 'lstm_base']
res = []
for model_name in model_names:
    if model_name in ['resnest50', 'resnet50']:
        for i in range(5):
            cur = np.load(os.path.join(res_path, "{}_fold_{}.npy".format(model_name, i)))
            res.append(cur[57886:][linux_test_order][:, linux_label_order])
    else:
        for i in range(5):
            res.append(np.load(os.path.join(res_path, "{}_fold_{}.npy".format(model_name, i)))[57886:])

In [75]:
test_names = np.load(os.path.join(feat_path, 'image_128_256.npz'))['test_names']

In [76]:
sub = pd.DataFrame()
sub['file_name'] = test_names
sub['label'] = np.argmax(np.mean(res, axis=0), axis=1)

In [77]:
sub['label'] = sub['label'].map({i:label for i, label in enumerate(labels)})

In [78]:
sub

,file_name,label
0,003gtit8kw.wav,one
1,006irl4pgx.wav,yes
2,007sh75o5w.wav,tree
3,009k6j5dbw.wav,three
4,009lyahcx8.wav,marvin
...,...,...
6830,zyvkhzi7pt.wav,off
6831,zzbo90jvjj.wav,nine
6832,zzgk3zkfr8.wav,right
6833,zzqta071j9.wav,three


In [79]:
import time
now = time.strftime("%Y%m%d_%H%M%S",time.localtime(time.time())) 
fname="submit_ensemble_" + now + ".csv"    
sub.to_csv(fname, index=False)